In [8]:
import numpy as np
import os
import time
from copy import copy
from itertools import combinations


In [5]:
n_nodes = 20
prob_matrix = np.random.uniform(0.0,0.01,(n_nodes,n_nodes))
prob_matrix


array([[1.74478429e-03, 8.78173006e-03, 5.46952346e-03, 3.97340599e-03,
        3.94953757e-03, 9.88258756e-03, 3.47076964e-03, 4.61239719e-03,
        8.14160775e-03, 2.02282380e-03, 4.81626722e-03, 8.33666701e-03,
        9.81688639e-03, 3.08726139e-03, 4.90226974e-03, 3.20373575e-03,
        6.23334657e-03, 3.63222226e-03, 1.64307087e-03, 6.03216574e-03],
       [7.92713782e-03, 1.81802474e-03, 3.98874656e-03, 7.41160155e-03,
        3.57790939e-03, 2.44678866e-03, 6.07720096e-04, 8.17757446e-03,
        1.49506539e-03, 6.07817327e-03, 6.46626514e-04, 8.99042970e-03,
        4.49759070e-03, 2.98715278e-03, 1.95176800e-03, 5.10718584e-03,
        3.71085553e-03, 3.44837356e-03, 5.19917556e-03, 1.50444492e-03],
       [7.95569348e-03, 3.09933034e-03, 3.61366191e-03, 4.90827979e-03,
        2.00009361e-04, 4.23203163e-03, 2.26359713e-03, 5.94769320e-04,
        7.51081094e-03, 3.09957156e-03, 9.62803818e-03, 9.58985325e-03,
        2.80458558e-03, 4.45352352e-03, 8.78212860e-03, 4.5267

In [9]:

class MonteCarloSampling(object):
    """
    Attributes
    --------
    edge_activations : edge activation functions matrix
    """
    def __init__(self, edge_activations):
        super().__init__()
        self.edge_activations = edge_activations
        self.n_nodes = edge_activations.shape[0]

    def simulate_episode(self, seeds, n_steps_max):
        """
        Simulates an episode where at each time step certain nodes activates

        Parameters
        --------

        seeds : initial set of seed nodes

        n_steps_max : number of time steps inside one episode
        """
        prob_matrix = self.edge_activations.copy()
        assert(seeds.shape[0]==self.n_nodes)
        history = np.array([seeds])

        active_nodes = seeds
        newly_active_nodes = active_nodes    #node active in the current timestep
        t = 0

        # Loop until either the time is exhausted or there is no new active node
        while(t<n_steps_max and np.sum(newly_active_nodes)>0):
            p = (prob_matrix.T*active_nodes).T  # This is the probability matrix but only with active nodes

            # Find edges exceeding an activation probability threshold and activate them
            activated_edges = p > np.random.rand(p.shape[0], p.shape[1])
            # Remove from the probability matrix the probability values related to the previously activated nodes
            prob_matrix = prob_matrix * ((p != 0) == activated_edges)

            # Activate those nodes which have at least and active edge and were not already active,
            # then add them to the currently active nodes
            newly_active_nodes = (np.sum(activated_edges, axis=0) > 0) * (1 - active_nodes)
            active_nodes = np.array(active_nodes + newly_active_nodes)

            history = np.concatenate((history, [newly_active_nodes]), axis=0)
            t += 1
        return history


    def mc_sampling(self, seeds, n_episodes, n_steps_max):
        """
        Implements Monte Carlo Sampling, from the edge probabilities and a given set of seeds, it returns 
        the node activation functions at each simulation

        Parameters
        --------
        seeds : set of seed nodes

        n_episode : number of monte carlo simulation

        n_steps_max : max number of steps in a episode
        """
        z = np.zeros(self.n_nodes)
        occurr_v_active = np.zeros(self.n_nodes) #occurencies of each node in all episodes

        estimated_prob_for_simulation = np.zeros(n_episodes) #this array contains nodes activation probabilities, one simulation per row
        
        for n in range(1,n_episodes+1):
            episode = self.simulate_episode(seeds=seeds, n_steps_max=n_steps_max)
            n_steps = episode.shape[0]
            
            for i in range(0,self.n_nodes): #occurency of each node at each episode
                if (len(np.argwhere(episode[:,i]==1))>0):  #this checks if node i activated in this episode
                    z[i]+=1
    
            estimated_prob = z/n #estimated probabilities up to simulation n
            estimated_prob_for_simulation.append(estimated_prob)

        return estimated_prob_for_simulation

In [14]:
budget = 5
montecarlo_simulations = 3
n_steps_max = 5

In [29]:
    sampler = MonteCarloSampling(prob_matrix)

    nodes = [d for d in range(0,n_nodes)]
    #seeds_combinations = list(combinations(nodes, self.budget)) #this take too much memory
    
    max_influence = 0
    best_seeds = np.zeros(n_nodes)
    sum_simulations = np.zeros(montecarlo_simulations) 

    for combination in combinations(nodes, budget): #enumerate all possible seeds given a budget
        seeds = np.zeros(n_nodes)
        seeds[list(combination)] = 1
        episode = sampler.simulate_episode(seeds, n_steps_max)

        print(episode)
        
        z = np.zeros(n_nodes)
        for i in range(0,n_nodes): #occurency of each node at each episode
                if (len(np.argwhere(episode[:,i]==1))>0):  #this checks if node i activated in this episode
                    z[i]+=1
        print(z)
        break
        

        #the best seed is the one where the sum of probabilities is the highest
        #total_sum = np.sum(nodes_probabilities_for_simulation, axis= 1)


[1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
